In [33]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch

tokenizer_base = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model_base = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForQuestionAnswering.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model.load_state_dict(torch.load('../model/model_weights_7.pth'))
model.eval()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [2]:
# model.save_pretrained('../model/qa_model_final')

In [3]:
# tokenizer.save_pretrained('../model/qa_tokenizer_final')

('../model/qa_tokenizer_final\\tokenizer_config.json',
 '../model/qa_tokenizer_final\\special_tokens_map.json',
 '../model/qa_tokenizer_final\\vocab.txt',
 '../model/qa_tokenizer_final\\added_tokens.json',
 '../model/qa_tokenizer_final\\tokenizer.json')

In [3]:
question = "Who announced the Novel Coronavirus SARS-CoV-2 COVID-19 as a pandemic outbreak?"
context = "The World Health Organization announced the Novel Coronavirus SARS-CoV-2 COVID-19 as a pandemic outbreak on the 11th of March, 20201. Since the WHO has warned about the beginning of the pandemic in March, basic protective equipment and safety protocols have not been always available in many medical institutions. healthcare workers are experiencing unprecedented pressure from stressors including but not limited to enormous workload, virus exposure, and inadequate PPE. The study followed a group of workers who were continuously and exclusively engaged in the treatment of patients with COVID-19 in Central Italy. The main stressors were prolonged periods of work in isolation, high workloads, compassion fatigue, and a lack of time for physical activity, meditation, or relaxation. Most of the workers who had contracted the infection were completely asymptomatic. SARS-CoV-2 machine learning smartphone application is vital in most outbreaks. Users learn how to perform valid tests through demonstration videos or blood sampling schematics included in the app. Use of the app by health authorities using fully anonymized location data could produce live disease maps. RDTs detected both total antibodies or IgG and IgM in blood, serum, or plasma."

In [3]:
import pandas as pd

df = pd.read_csv('../clean.csv')
df_papers = df.groupby('paper_id')['text'].sum().reset_index()

In [22]:
df_papers["text"][0]

'according to current live statistics at the time of editing this letter, Russia has been the third country in the world to be affected by COVID-19 with both new cases and death rates rising. it remains in a position of advantage due to the later onset of the viral spread within the country since the worldwide disease outbreak.the first step in fighting the epidemic was nationwide lock down on March 30 th , 2020.most of the multidisciplinary hospitals have been repurposed as dedicated COVID-19 centres, so the surgeons started working as infectious disease specialists. such a reallocation of health care capacity results in the effective management of this epidemiological problem 1 . the staff has undergone on-line 36-hour training course to become qualified in coronavirus infection treatment.the surgeons of COVID-19 dedicated hospitals do rarely practice surgery. when ICU patients need mechanical ventilation, percutaneous tracheostomy under endoscopic control is mostly performed, as it 

In [30]:
context = df_papers["text"][0]
question = "What is the surgery that might cause a worldwide healthcare catastrophe in the post-pandemic era 5?"

In [28]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
result = question_answerer(question=question, context=context)
result['answer']

'WHO has warned about the beginning of the pandemic'

In [29]:
import pandas as pd
questions_df = pd.read_csv('../comparison.csv')
def inference(context, question):
    result = question_answerer(question=question, context=context)
    return result['answer']

questions_df['model_output'] = questions_df.apply(lambda row: inference(row['context'], row['question']), axis=1)
questions_df

,question,context,correct,baseline_output,model_output
0,Who announced the Novel Coronavirus SARS-CoV-2...,The World Health Organization announced the No...,The World Health Organization,"about the beginning of the pandemic in March, ...",WHO has warned about the beginning of the pand...
1,What are not always available in many medical ...,The World Health Organization announced the No...,basic protective equipment,"about the beginning of the pandemic in March, ...",announced the Novel Coronavirus SARS-CoV-2 COVID
2,What are the main stressors?,The World Health Organization announced the No...,"prolonged periods of work in isolation, high w...",in the app. Use of the app by,Since the WHO has warned about the beginning o...
3,What is the most common cause of death in dogs?,Cancer is frequent in dogs and is by far their...,Cancer,coming years. The introduction of,in advance
4,How many dogs are newly diagnosed with cancer ...,Cancer is frequent in dogs and is by far their...,46 million,coming years. The introduction of,in advance


In [30]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch

model_base = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
import pandas as pd
questions_df = pd.read_csv('../comparison.csv')
def inference(context, question):
    question_answerer = pipeline("question-answering", model=model_base, tokenizer=tokenizer_base)
    result = question_answerer(question=question, context=context)
    return result['answer']

questions_df['baseline_output'] = questions_df.apply(lambda row: inference(row['context'], row['question']), axis=1)
questions_df

,question,context,correct,baseline_output,model_output
0,Who announced the Novel Coronavirus SARS-CoV-2...,The World Health Organization announced the No...,The World Health Organization,COVID-19 in Central Italy. The main stressors,World Health
1,What are not always available in many medical ...,The World Health Organization announced the No...,basic protective equipment,COVID-19 in Central Italy. The main stressors,medical institutions
2,What are the main stressors?,The World Health Organization announced the No...,"prolonged periods of work in isolation, high w...",COVID-19 as a pandemic,main stressors
3,What is the most common cause of death in dogs?,Cancer is frequent in dogs and is by far their...,Cancer,pandemic. college,Cancer
4,How many dogs are newly diagnosed with cancer ...,Cancer is frequent in dogs and is by far their...,46 million,pandemic. college,46 million dogs


In [35]:
questions_df.to_csv('../comparison.csv', index=False)